In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df = pd.read_csv('/content/drive/MyDrive/congress_tweets.csv')
df.head()

,Unnamed: 0,tweet,target
0,0,spending shocker cbo latest debt projection sh...,0
1,1,will you vote for congress in 2024 congress co...,1
2,2,triviapotus we need termlimits for congress,1
3,3,eam s jaishankar hit out at congress for quad ...,0
4,4,is this why congress supported amp celebrated ...,1


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [4]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-ormlyebp
  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-ormlyebp
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 9ca68d37027af9f6a30d54640347ce3b2e2694b3
  Preparing metadata (setup.py) ... done
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=7620 sha256=ebc38bc990638a4a95b9ebf7106dc7fb55acb2ae267ea00975caad42e2d84ce4
  Stored in directory: /tmp/pip-ephem-wheel-cache-fj9js720/wheels/5c/94/34/99d5ff65e88b8d9a6c5e8d8652f2311d87790a61a1b7466e21
Successfully built preprocess-kgptalkie


In [5]:
import preprocess_kgptalkie as ps
import re
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [6]:
df['tweet'] = df['tweet'].apply(lambda x: get_clean(x))
df.head()

,Unnamed: 0,tweet,target
0,0,spending shocker cbo latest debt projection sh...,0
1,1,will you vote for congress in 2024 congress co...,1
2,2,triviapotus we need termlimits for congress,1
3,3,eam s jaishankar hit out at congress for quad ...,0
4,4,is this why congress supported amp celebrated ...,1


In [7]:
tfidf = TfidfVectorizer(max_features=10000)
X = df['tweet']
y = df['target']

X = tfidf.fit_transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 0)

In [9]:
clf = LinearSVC()
clf.fit(X_train, y_train)

LinearSVC()

In [10]:
y_pred = clf.predict(X_test)

In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.77      0.78      2831
           1       0.80      0.82      0.81      3220

    accuracy                           0.79      6051
   macro avg       0.79      0.79      0.79      6051
weighted avg       0.79      0.79      0.79      6051



In [16]:
x = 'Recent debates in Indian politics have focused on the economic reforms and their impact on various sectors of the country'
x = get_clean(x)
vec = tfidf.transform([x])

In [19]:
state = clf.predict(vec)
if state == 0:
  print("Negative")
else:
  print("Positive")

Negative
